In [13]:
%load_ext autoreload
%autoreload

In [14]:
from dipy.data import read_stanford_labels
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
hardi_img, gtab, labels_img = read_stanford_labels();
data = hardi_img.get_data();
labels = labels_img.get_data();
affine = hardi_img.affine

In [6]:
print(type(hardi_img))
print(type(gtab))
print(type(labels_img))

<class 'nibabel.nifti1.Nifti1Image'>
<class 'dipy.core.gradients.GradientTable'>
<class 'nibabel.nifti1.Nifti1Image'>


In [16]:
white_matter = (labels == 1) | (labels == 2)

In [17]:
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model

csa_model = CsaOdfModel(gtab, sh_order=6)
csa_peaks = peaks_from_model(csa_model, data, default_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=45,
                             mask=white_matter)

In [18]:
from dipy.tracking.local import ThresholdTissueClassifier

classifier = ThresholdTissueClassifier(csa_peaks.gfa, .25)

In [19]:
from dipy.tracking import utils

seed_mask = labels == 2
seeds = utils.seeds_from_mask(seed_mask, density=[2, 2, 2], affine=affine)

In [20]:
from dipy.tracking.local import LocalTracking
from dipy.viz import fvtk
from dipy.viz.colormap import line_colors

# Initialization of LocalTracking. The computation happens in the next step.
streamlines = LocalTracking(csa_peaks, classifier, seeds, affine, step_size=.5)

# Compute streamlines and store as a list.
streamlines = list(streamlines)

# Prepare the display objects.
color = line_colors(streamlines)
if fvtk.have_vtk:
    streamlines_actor = fvtk.line(streamlines, line_colors(streamlines))

    # Create the 3D display.
    r = fvtk.ren()
    fvtk.add(r, streamlines_actor)

    # Save still images for this static example. Or for interactivity use
    # fvtk.show
    fvtk.show(r)#.record(r, n_frames=1, out_path='./deterministic.png',
                #size=(800, 800))